<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AAI_VECTORDB_DEMO_LLM2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faiss-cpu google-generativeai numpy -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 28.4 MB/s eta 0:00:00


In [3]:
import os
import faiss
import numpy as np
import google.generativeai as genai

# --- 1. Your provided Google Gemini API Configuration ---
class AgentConfig:
    LLM_MODEL_NAME: str = "gemini-1.5-flash"
    EMBEDDING_MODEL_NAME: str = "models/text-embedding-004"

GOOGLE_API_KEY = None
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GEMINI')
    print("Google Generative AI configured successfully using Colab Secrets.")
except (ImportError, KeyError):
    print("Not running in Google Colab or 'GEMINI' secret not found. Attempting to get 'GEMINI' environment variable.")
    GOOGLE_API_KEY = os.getenv('GEMINI')

if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print(f"Gemini API configured with model: {AgentConfig.LLM_MODEL_NAME}")
else:
    print("Warning: GOOGLE_API_KEY not found. LLM calls will not work.")
    print("Please set your 'GEMINI' environment variable or Colab secret.")
    exit()

# --- 2. Vector Database Setup with Gemini Embeddings ---
def get_embeddings(texts):
    try:
        response = genai.embed_content(
            model=AgentConfig.EMBEDDING_MODEL_NAME,
            content=texts,
            task_type="retrieval_document"
        )
        return np.array(response['embedding'])
    except Exception as e:
        print(f"An error occurred while generating embeddings: {e}")
        return None

documents = [
    "The sky is a brilliant blue today.",
    "A dog is a man's best friend.",
    "The weather is sunny and warm.",
    "I have a furry golden retriever.",
    "The latest iPhone has a great camera.",
    "I love spending time with my pets.",
    "This new smartphone has an impressive screen.",
    "The sun is shining brightly.",
    "The capital of France is Paris.",
    "The Eiffel Tower is located in Paris.",
    "Mars is known as the Red Planet.",
    "Water is a chemical compound with the formula H2O."
]

print("\nGenerating embeddings for the knowledge base...")
document_embeddings = get_embeddings(documents)

if document_embeddings is None:
    print("Failed to create embeddings. Exiting.")
    exit()

d = document_embeddings.shape[1]
print(f"Embeddings have dimension: {d}")

index = faiss.IndexFlatL2(d)
index.add(document_embeddings)
print(f"FAISS index created with {index.ntotal} documents.")


# --- 3. RAG and Tool-Use Functions ---
def get_relevant_documents(query, index, k=3):
    """
    A "tool" function for the agent to retrieve relevant documents from the vector DB.
    """
    try:
        query_embedding_response = genai.embed_content(
            model=AgentConfig.EMBEDDING_MODEL_NAME,
            content=[query],
            task_type="retrieval_query"
        )
        query_embedding = np.array(query_embedding_response['embedding'])
        distances, indices = index.search(query_embedding, k)
        relevant_docs = [documents[i] for i in indices[0]]
        return relevant_docs
    except Exception as e:
        print(f"An error occurred during retrieval: {e}")
        return []

def get_direct_answer(query):
    """
    A "tool" function for the agent to get a direct answer from the LLM.
    """
    try:
        model = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
        response = model.generate_content(query)
        return response.text
    except Exception as e:
        return f"An error occurred: {e}"

# --- 4. Agentic Pipeline ---
def agentic_rag_pipeline(query):
    """
    The agentic pipeline that first plans and then executes.
    """
    print(f"\n--- Agent receives query: '{query}' ---")

    # Step 1: Agentic Planning
    # The agent uses a prompt to decide which action to take.
    planner_prompt = f"""
    You are an intelligent agent that decides whether to perform a search or answer a question directly.
    You have access to a small knowledge base via a search tool.

    If the question is about specific facts that might be in the knowledge base, choose 'SEARCH'.
    The knowledge base contains information about:
    - dogs and pets
    - the weather
    - specific phones
    - facts about Paris and France
    - basic chemistry (H2O)
    - planets (Mars)

    If the question is general knowledge, creative, or conversational, choose 'ANSWER'.
    For example, 'write a poem' or 'who is the president of the US?'.

    Your response must be a single word: either 'SEARCH' or 'ANSWER'.

    Question: {query}
    Choice:
    """

    try:
        model = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
        planning_response = model.generate_content(planner_prompt)
        decision = planning_response.text.strip().upper()
    except Exception as e:
        print(f"Agent planning failed: {e}. Defaulting to SEARCH.")
        decision = "SEARCH"

    print(f"Agent decision: {decision}")

    # Step 2: Tool Execution based on the decision
    if decision == 'SEARCH':
        # Execute the retrieval tool (vector DB)
        relevant_docs = get_relevant_documents(query, index=index)

        print("\n--- Agent uses 'SEARCH' tool ---")
        for i, doc in enumerate(relevant_docs):
            print(f"  - Retrieved document {i+1}: {doc}")

        # Step 3: Construct a RAG prompt and get the final answer
        context = "\n".join(relevant_docs)
        if not context:
            context = "No relevant information was found in the knowledge base."

        rag_prompt = f"""
        You are a helpful assistant. Use the following context to answer the question.
        If the answer is not in the context, state that you don't know based on the provided information.

        Context:
        {context}

        Question: {query}

        Answer:
        """
        final_answer = model.generate_content(rag_prompt).text

    elif decision == 'ANSWER':
        # Execute the direct answer tool (LLM without context)
        print("\n--- Agent uses 'ANSWER' tool ---")
        final_answer = get_direct_answer(query)

    else:
        # Fallback for unexpected decisions
        print("\n--- Agent had an unexpected decision, defaulting to direct answer. ---")
        final_answer = get_direct_answer(query)

    print("\n--- Final Answer ---")
    print(final_answer)


# --- 5. Demo the Agentic Pipeline ---
if __name__ == "__main__":

    # Example 1: A specific question that should trigger 'SEARCH'
    user_question_1 = "What is the capital of France?"
    agentic_rag_pipeline(user_question_1)

    print("\n" + "="*80 + "\n")

    # Example 2: A question that should trigger 'SEARCH'
    user_question_2 = "Tell me about my pets."
    agentic_rag_pipeline(user_question_2)

    print("\n" + "="*80 + "\n")

    # Example 3: A question that is general knowledge and should trigger 'ANSWER'
    user_question_3 = "What is the largest planet in our solar system?"
    agentic_rag_pipeline(user_question_3)

    print("\n" + "="*80 + "\n")

    # Example 4: A creative question that should trigger 'ANSWER'
    user_question_4 = "Write a short poem about the sun."
    agentic_rag_pipeline(user_question_4)

Google Generative AI configured successfully using Colab Secrets.
Gemini API configured with model: gemini-1.5-flash

Generating embeddings for the knowledge base...
Embeddings have dimension: 768
FAISS index created with 12 documents.

--- Agent receives query: 'What is the capital of France?' ---
Agent decision: SEARCH

--- Agent uses 'SEARCH' tool ---
  - Retrieved document 1: The capital of France is Paris.
  - Retrieved document 2: The Eiffel Tower is located in Paris.
  - Retrieved document 3: Mars is known as the Red Planet.

--- Final Answer ---
Paris




--- Agent receives query: 'Tell me about my pets.' ---
Agent decision: SEARCH

--- Agent uses 'SEARCH' tool ---
  - Retrieved document 1: I love spending time with my pets.
  - Retrieved document 2: I have a furry golden retriever.
  - Retrieved document 3: A dog is a man's best friend.

--- Final Answer ---
Based on the provided text, you have a furry golden retriever.




--- Agent receives query: 'What is the largest planet